In [1]:
import torch
from torch.autograd import Variable, Function
from torch import nn

In [11]:
# 三个步骤：定义模型，梯度回传，更新模型
## 1. 定义模型有两种方法
D_in, H, D_out = 100, 10, 100
model1 = nn.Sequential(
    nn.Linear(D_in, H),
    nn.ReLU(),
    nn.Linear(H, D_out),
)

class CModel(nn.Module):
    def __init__(self, d_in, h, d_out):
        super(CModel, self).__init__()
        self.input_layer = nn.Linear(d_in, h)
        self.hidden = nn.ReLU()
        self.output = nn.Linear(h, d_out)
    def forward(self, x_in):
        x = self.input_layer(x_in)
        x = self.hidden(x)
        x = self.output(x)
        return x
    
model2 = CModel(D_in, H, D_out)
model2.parameters = model1.parameters
# prepare data
x = torch.randn((10, 100))
x1 = Variable(x, requires_grad=True).clone()
x2 = Variable(x, requires_grad=True).clone()
y_pred_1 = model1(x1)
y_pred_2 = model2(x2)

In [ ]:
# 图结构控制：动态多层和参数共享
class DynamicNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        In the constructor we construct three nn.Linear instances that we will use
        in the forward pass.
        """
        super(DynamicNet, self).__init__()
        self.input_linear = torch.nn.Linear(D_in, H)
        self.middle_linear = torch.nn.Linear(H, H)
        self.output_linear = torch.nn.Linear(H, D_out)

    def forward(self, x):
        """
        For the forward pass of the model, we randomly choose either 0, 1, 2, or 3
        and reuse the middle_linear Module that many times to compute hidden layer
        representations.

        Since each forward pass builds a dynamic computation graph, we can use normal
        Python control-flow operators like loops or conditional statements when
        defining the forward pass of the model.

        Here we also see that it is perfectly safe to reuse the same Module many
        times when defining a computational graph. This is a big improvement from Lua
        Torch, where each Module could be used only once.
        """
        h_relu = self.input_linear(x).clamp(min=0)
        for _ in range(random.randint(0, 3)):
            h_relu = self.middle_linear(h_relu).clamp(min=0)
        y_pred = self.output_linear(h_relu)
        return y_pred
    
# PS: 这里，中间线性层进行了多次迭代（迭代次数由随机数决定），并且进行了参数共享（中间层的重复，用的是同一个参数矩阵）